## IMPORT

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 462 kB 18.4 MB/s 
     |████████████████████████████████| 4.2 MB 9.9 MB/s 


In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [ ]:
cd /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0 

/content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0


In [ ]:
print(os.getcwd())

/content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0


## Preparing the dataset

In [ ]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incompleteFR71ZZ/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


In [ ]:
import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incompleteR1B2LF/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.


In [ ]:
ratings = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    "user_id": x["user_id"],
    "add" : x["add"]
})
locations = wishEmbeddingDs.map(lambda x: x["location_name"])

In [ ]:
for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'add': 0, 'location_name': b'JAKARTA', 'user_id': b'93'}
{'add': 0, 'location_name': b'DUBAI', 'user_id': b'112'}


In [ ]:
tf.random.set_seed(42)

NUM_DATA = ratings.__len__().numpy()

shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## MODEL

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add")

    user_embeddings, location_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 6s 69ms/step - root_mean_squared_error: 0.4572 - factorized_top_k/top_1_categorical_accuracy: 0.0014 - factorized_top_k/top_5_categorical_accuracy: 0.0373 - factorized_top_k/top_10_categorical_accuracy: 0.0787 - factorized_top_k/top_50_categorical_accuracy: 0.4046 - factorized_top_k/top_100_categorical_accuracy: 0.8098 - loss: 0.2086 - regularization_loss: 0.0000e+00 - total_loss: 0.2086
Epoch 2/3
43/43 [==============================] - 3s 69ms/step - root_mean_squared_error: 0.4461 - factorized_top_k/top_1_categorical_accuracy: 0.0029 - factorized_top_k/top_5_categorical_accuracy: 0.0390 - factorized_top_k/top_10_categorical_accuracy: 0.0827 - factorized_top_k/top_50_categorical_accuracy: 0.4101 - factorized_top_k/top_100_categorical_accuracy: 0.8152 - loss: 0.1988 - regularization_loss: 0.0000e+00 - total_loss: 0.1988
Epoch 3/3
22/22 [==============================] - 3s 61ms/step - root_mean_squared_error: 0.4462 - factorized_top_k

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [ ]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=5)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/5
43/43 [==============================] - 6s 113ms/step - root_mean_squared_error: 0.5008 - factorized_top_k/top_1_categorical_accuracy: 0.0065 - factorized_top_k/top_5_categorical_accuracy: 0.1628 - factorized_top_k/top_10_categorical_accuracy: 0.2235 - factorized_top_k/top_50_categorical_accuracy: 0.5607 - factorized_top_k/top_100_categorical_accuracy: 0.8430 - loss: 3175.8793 - regularization_loss: 0.0000e+00 - total_loss: 3175.8793
Epoch 2/5
43/43 [==============================] - 6s 135ms/step - root_mean_squared_error: 0.5001 - factorized_top_k/top_1_categorical_accuracy: 0.0432 - factorized_top_k/top_5_categorical_accuracy: 0.5049 - factorized_top_k/top_10_categorical_accuracy: 0.6199 - factorized_top_k/top_50_categorical_accuracy: 0.8138 - factorized_top_k/top_100_categorical_accuracy: 0.9569 - loss: 3175.7796 - regularization_loss: 0.0000e+00 - total_loss: 3175.7796
Epoch 3/5
43/43 [==============================] - 7s 160ms/step - root_mean_squared_error: 0.5004 - f

In [ ]:
model.evaluate(cached_test, return_dict=True)

22/22 [==============================] - 1s 62ms/step - root_mean_squared_error: 0.5016 - factorized_top_k/top_1_categorical_accuracy: 0.0288 - factorized_top_k/top_5_categorical_accuracy: 0.0911 - factorized_top_k/top_10_categorical_accuracy: 0.1483 - factorized_top_k/top_50_categorical_accuracy: 0.8386 - factorized_top_k/top_100_categorical_accuracy: 0.9953 - loss: 1343.8418 - regularization_loss: 0.0000e+00 - total_loss: 1343.8418


{'factorized_top_k/top_100_categorical_accuracy': 0.9952641129493713,
 'factorized_top_k/top_10_categorical_accuracy': 0.14826957881450653,
 'factorized_top_k/top_1_categorical_accuracy': 0.02877959981560707,
 'factorized_top_k/top_50_categorical_accuracy': 0.8386156558990479,
 'factorized_top_k/top_5_categorical_accuracy': 0.09107468277215958,
 'loss': 540.1198120117188,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.5016109943389893,
 'total_loss': 540.1198120117188}

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((locations.batch(100), locations.batch(100).map(model.location_model)))
)

# Get recommendations.
user_id = "45"
_, titles = index(tf.constant([user_id]))
print(f"Recommendations for Rolas : {titles[0, :5]}")

Recommendations for Rolas : [b'INDONESIA' b'TEGAL' b'MEKKAH' b'SLOVENIA' b'PRANCIS']


### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 5s 85ms/step - root_mean_squared_error: 0.4567 - factorized_top_k/top_1_categorical_accuracy: 2.7322e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0211 - factorized_top_k/top_10_categorical_accuracy: 0.0523 - factorized_top_k/top_50_categorical_accuracy: 0.3571 - factorized_top_k/top_100_categorical_accuracy: 0.7973 - loss: 3181.1915 - regularization_loss: 0.0000e+00 - total_loss: 3181.1915
Epoch 2/3
43/43 [==============================] - 3s 77ms/step - root_mean_squared_error: 0.4455 - factorized_top_k/top_1_categorical_accuracy: 2.7322e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0313 - factorized_top_k/top_10_categorical_accuracy: 0.0744 - factorized_top_k/top_50_categorical_accuracy: 0.4341 - factorized_top_k/top_100_categorical_accuracy: 0.8550 - loss: 3173.7099 - regularization_loss: 0.0000e+00 - total_loss: 3173.7099
Epoch 3/3
22/22 [==============================] - 2s 70ms/step - root_mean_squared_error: 0.444